In [35]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

load_dotenv()

True

In [31]:
model = ChatOpenAI()
parser = StrOutputParser()

In [ ]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluation_score: float

In [22]:
def generate_outline(state: BlogState) -> BlogState:
    # Extract title from the state
    title = state['title']
    # Create a prompt
    prompt = f"Generate a detailed outline for a blog post titled: {title}"
    # Invoke the LLM
    response = model.invoke(prompt)
    # Update the state with the generated outline
    state['outline'] = response.content
    return state

In [23]:
def generate_blog(state: BlogState) -> BlogState:
    # Extract title and outline from the state
    title = state['title']
    outline = state['outline']
    
    # Create a prompt 
    prompt = f'Generate a detailed blog on the title {title} using the following outline:\n{outline}'
    
    # Invoke the LLM
    response = model.invoke(prompt)
    # Update the state with the generated content
    state['content'] = response.content
    return state

In [48]:
def evaluate_blog(state: BlogState) -> BlogState:
    # Extract content from the state
    content = state['content']
    
    # Create a prompt for evaluation
    prompt = PromptTemplate(template="Evaluate the following blog content:\n{content}\nGive a total score on a scale of 1 to 10 based on clarity, engagement, and informativeness. Only provide the total score, noting else.",
                            input_variables=["content"])
    
    # Create a chain
    chain = prompt | model | parser
    
    # Invoke the LLM
    response = chain.invoke({"content": content})
    
    # Update the state with the evaluation score
    state['evaluation_score'] = float(response.strip())
    return state

In [49]:
# Define the state graph
graph = StateGraph(BlogState)

# Add nodes to the graph
graph.add_node('generate_outline', generate_outline)
graph.add_node('generate_blog', generate_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# Add edges to connect the nodes
graph.add_edge(START, 'generate_outline')
graph.add_edge('generate_outline', 'generate_blog')
graph.add_edge('generate_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

# Compile the graph
workflow = graph.compile()

In [50]:
# Invoke the graph with an initial state
initial_state = BlogState(title='Rise of AI in Sweden', outline='', content='')
final_state = workflow.invoke(initial_state)
print("Final State:", final_state)

Final State: {'title': 'Rise of AI in Sweden', 'outline': "I. Introduction\n    A. Explanation of AI and its significance\n    B. Overview of Sweden's growing investment in AI technology\n    C. Preview of key points to be discussed in the blog post\n\nII. History of AI in Sweden\n    A. Early adoption of AI technology in Swedish industries\n    B. Government initiatives to promote AI research and development\n    C. Established AI companies in Sweden\n\nIII. Current AI Landscape in Sweden\n    A. Overview of AI startups and research institutions in Sweden\n    B. Industries leveraging AI technology in Sweden\n    C. Impact of AI on the Swedish economy and society\n\nIV. Government Support for AI in Sweden\n    A. Funding and grants for AI research and development projects\n    B. Policies and regulations promoting the responsible use of AI technology\n    C. Collaboration between government, academia, and industry in advancing AI in Sweden\n\nV. Challenges and Opportunities\n    A. Et

In [51]:
print("Generated outline:\n\n", final_state['outline'])

Generated outline:

 I. Introduction
    A. Explanation of AI and its significance
    B. Overview of Sweden's growing investment in AI technology
    C. Preview of key points to be discussed in the blog post

II. History of AI in Sweden
    A. Early adoption of AI technology in Swedish industries
    B. Government initiatives to promote AI research and development
    C. Established AI companies in Sweden

III. Current AI Landscape in Sweden
    A. Overview of AI startups and research institutions in Sweden
    B. Industries leveraging AI technology in Sweden
    C. Impact of AI on the Swedish economy and society

IV. Government Support for AI in Sweden
    A. Funding and grants for AI research and development projects
    B. Policies and regulations promoting the responsible use of AI technology
    C. Collaboration between government, academia, and industry in advancing AI in Sweden

V. Challenges and Opportunities
    A. Ethical considerations surrounding AI technology in Sweden
  

In [52]:
blog_content = final_state['content']
print("Generated blog content:\n\n", blog_content)

Generated blog content:

 I. Introduction

Artificial Intelligence (AI) is a transformative technology that is revolutionizing industries and societies across the world. In Sweden, there has been a significant rise in investment and interest in AI technology, positioning the country as a leader in the AI landscape. In this blog post, we will explore the history of AI in Sweden, the current AI landscape, government support for AI initiatives, challenges and opportunities, case studies of successful AI implementations, and the future outlook for AI in Sweden.

II. History of AI in Sweden

Sweden has been an early adopter of AI technology, with industries such as automotive, healthcare, and finance leading the way in implementing AI solutions. The Swedish government has also played a crucial role in promoting AI research and development through various initiatives and collaborations. Established AI companies in Sweden, such as Peltarion and Element AI, have been driving innovation in the 

In [53]:
print("Evaluation Score:", final_state['evaluation_score'])

Evaluation Score: 9.0
